# TAM HTML to page by page Unicode HTML
### Code by Ravi Annaswamy October 2019
### Tested first on Keeladi Research Report from Tamilnadu Archaeology Department.

# Step 1.  Using pdftohtml.net HTML, first create the html version of the PDF.

### from this great web service.
### https://www.pdftohtml.net/

### Later, we will find a way to do it without going to the webservice.

In [1]:
html_file = './Keeladi_Document_Tamil/Keeladi_Document_Tamil_p2h.html'

In [2]:
with open(html_file, encoding='utf-8') as f:
    text=f.read()

In [13]:
text[5000:5500]

'ly:Arial,serif;font-size:50.0px;color:rgb(0,0,0);font-weight:normal;font-style:normal;text-decoration: none}\nspan.cls_022{font-family:Arial,serif;font-size:12.1px;color:rgb(0,0,0);font-weight:normal;font-style:normal;text-decoration: none}\ndiv.cls_022{font-family:Arial,serif;font-size:12.1px;color:rgb(0,0,0);font-weight:normal;font-style:normal;text-decoration: none}\nspan.cls_021{font-family:Arial,serif;font-size:39.4px;color:rgb(87,88,91);font-weight:normal;font-style:normal;text-decoration: no'

# Step 2.  From html source code, identify how many font settings are being used. 
### This may run into thousands.
### Detect how many are to be left untranslated (English) and which ones are TAM encoded.

In [67]:
import re

html2 = text

def get_classname(cno):
    c=str(cno)
    if len(c)==1:
        return 'cls_00'+c
    if len(c)==2:
        return 'cls_0'+c
    if len(c)>=3:
        return 'cls_'+c

    
eng_classes=[]
tam_classes=[]
class_count=0
for cno in range(5000):
    cname=get_classname(cno)
    pattern = '<span class="'+cname+'">(.*?)</span>'
    #print(pattern)
    matches= re.findall(pattern,html2)
    if matches:
        all_text=''.join(matches)
        alpha=''.join(list(sorted(set(all_text))))
        for c in '‘’“”':
            alpha=alpha.replace(c,'')
        #print(cname, alpha, len(matches), ord(min(alpha)), ord(max(alpha))) # T.to_unicode(matches[0]))
        if cno>class_count:
            class_count=cno
        if ord(max(alpha))<220:
            eng_classes.append((cno,ord(max(alpha))))
        else:
            tam_classes.append((cno,ord(max(alpha))))
            
class_count+=1
print("class_count:", class_count)
print('Eng classes:', len(eng_classes))
print('Tam classes:', len(tam_classes))


class_count: 1369
Eng classes: 76
Tam classes: 1287


### For debugging if a font that is not tamil, is getting interpreted as Tamil.
#### Find its class number and plug it in here.
#### Check if a particular class in tamil is truly an english if you get bad mapping

In [64]:
print([(classno,count) for classno,count in tam_classes if classno==6])

[(6, 257)]


### If you still see some classes that are getting misinterpreted as Tamil, you need to increase the threshold by looking through this.

In [65]:
print('Tamil classes with low-code letters:', sorted(tam_classes, key=lambda k:k[1])[:10])

Tamil classes with low-code letters: [(405, 220), (1257, 220), (561, 222), (863, 222), (943, 222), (1158, 222), (21, 238), (286, 238), (365, 238), (516, 238)]


# Step 3. Actual conversion step. 
### Locate tamil phrases and sentences and paras and translate using our TAM decoder.

In [68]:
import TAM 

T = TAM.TAM()

html2 = text

P1 = '<span class="'
P2 = '">'
P3 = '(.*?)'
P4 = '</span>'
eng_class_list=[cno for cno,cnt in eng_classes]

debug=False
i=0
for cno in range(class_count):
    #print(get_classname(cno))
    if cno not in eng_class_list:
        pattern = pattern_left + get_classname(cno) + pattern_right
        matches= re.findall(pattern,html2)
        for m in matches:
            html2=html2.replace(P1 + get_classname(cno)+ P2 + m + P4, \
                                P1 + get_classname(cno)+ P2 +T.to_unicode(m)+ P4)
            if debug:
                print(m)
                print(P1 + get_classname(cno)+ P2 + m + P4)
                print(P1 + get_classname(cno)+ P2 +T.to_unicode(m)+ P4)
                print()
                i+=1
                if i==10: break
    if debug:
        if i==10: break

319 TAM to Unicode replacements read!


# Step 4. Write out the Unicode HTML.

In [62]:
outfile = html_file.replace('.html','_pagewise_unicode.html')
with open(outfile,'w', encoding='utf-8') as f:
    f.write(html2)

## Done!